In [7]:
import pandas as pd
import urllib.request
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [8]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [12]:
data = pd.read_csv('drive/MyDrive/Colab Notebooks/abcnews-date-text.csv')

In [14]:
text = data[['headline_text']]
text.head(5)
text['headline_text'] = text.apply(lambda row: nltk.word_tokenize(row['headline_text']), axis=1)

<ipython-input-14-71ca6d40005f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text['headline_text'] = text.apply(lambda row: nltk.word_tokenize(row['headline_text']), axis=1)


In [15]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
text['headline_text'] = text['headline_text'].apply(lambda x: [word for word in x if word not in (stop)])
print(text.head(5))

                                       headline_text
0   [aba, decides, community, broadcasting, licence]
1    [act, fire, witnesses, must, aware, defamation]
2     [g, calls, infrastructure, protection, summit]
3          [air, nz, staff, aust, strike, pay, rise]
4  [air, nz, strike, affect, australian, travellers]


<ipython-input-15-3a72255956f2>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text['headline_text'] = text['headline_text'].apply(lambda x: [word for word in x if word not in (stop)])


In [16]:
from nltk.stem import WordNetLemmatizer
text['headline_text'] = text['headline_text'].apply(lambda x: [WordNetLemmatizer().lemmatize(word, pos='v') for word in x])
print(text.head(5))

                                       headline_text
0       [aba, decide, community, broadcast, licence]
1      [act, fire, witness, must, aware, defamation]
2      [g, call, infrastructure, protection, summit]
3          [air, nz, staff, aust, strike, pay, rise]
4  [air, nz, strike, affect, australian, travellers]


<ipython-input-16-a1ec0cfcf7fa>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text['headline_text'] = text['headline_text'].apply(lambda x: [WordNetLemmatizer().lemmatize(word, pos='v') for word in x])


In [17]:
# 길이 3이하 단어 제거
tokenized_doc = text['headline_text'].apply(lambda x: [word for word in x if len(word) > 3])
print(tokenized_doc[:5])

0       [decide, community, broadcast, licence]
1      [fire, witness, must, aware, defamation]
2    [call, infrastructure, protection, summit]
3                   [staff, aust, strike, rise]
4      [strike, affect, australian, travellers]
Name: headline_text, dtype: object


In [18]:
# 역토큰화 (토큰화 작업을 되돌림)
detokenized_doc = []
for i in range(len(text)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)

# 다시 text['headline_text']에 재저장
text['headline_text'] = detokenized_doc

<ipython-input-18-93127686f901>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text['headline_text'] = detokenized_doc


In [19]:
# 상위 1,000개의 단어를 보존
vectorizer = TfidfVectorizer(stop_words='english', max_features= 1000)
X = vectorizer.fit_transform(text['headline_text'])

# TF-IDF 행렬의 크기 확인
print('TF-IDF 행렬의 크기 :',X.shape)

TF-IDF 행렬의 크기 : (1082168, 1000)


In [20]:
vectorizer.vocabulary_

{'decide': 255,
 'community': 191,
 'licence': 521,
 'witness': 981,
 'infrastructure': 457,
 'protection': 678,
 'summit': 858,
 'staff': 833,
 'aust': 59,
 'strike': 847,
 'rise': 745,
 'affect': 16,
 'australian': 61,
 'jump': 486,
 'record': 707,
 'break': 112,
 'aussie': 57,
 'waste': 962,
 'match': 551,
 'address': 13,
 'security': 782,
 'council': 213,
 'iraq': 469,
 'australia': 60,
 'million': 563,
 'celebrate': 150,
 'plan': 629,
 'ahead': 22,
 'boost': 105,
 'water': 964,
 'supply': 860,
 'unite': 936,
 'state': 838,
 'dismiss': 274,
 'report': 724,
 'troop': 925,
 'british': 116,
 'arrive': 48,
 'lead': 508,
 'double': 279,
 'bushfire': 125,
 'victims': 946,
 'urge': 943,
 'businesses': 127,
 'prepare': 656,
 'attack': 54,
 'final': 342,
 'defeat': 258,
 'fuel': 371,
 'leave': 515,
 'miss': 567,
 'fund': 372,
 'bank': 74,
 'home': 429,
 'help': 416,
 'youth': 996,
 'chief': 160,
 'fail': 321,
 'secure': 781,
 'councillor': 214,
 'protect': 677,
 'heritage': 417,
 'welcome':

In [28]:
from sklearn.decomposition import LatentDirichletAllocation# 사이킷런에는 그게 구현이 되어져있다.

In [29]:
lda_model = LatentDirichletAllocation(n_components=10, learning_method='online', random_state=777, max_iter=1)

In [30]:
lda_top = lda_model.fit_transform(X)

In [31]:
terms = vectorizer.get_feature_names_out()   # 모든 피쳐들이 나오게 된다. 1000개 가량이다.
terms

array(['2013', '2014', '2015', '2016', 'abbott', 'aboriginal', 'abuse',
       'accc', 'accept', 'access', 'accident', 'accuse', 'action',
       'address', 'adelaide', 'admit', 'affect', 'afghan', 'afghanistan',
       'africa', 'agree', 'agreement', 'ahead', 'airport', 'alcohol',
       'alert', 'alice', 'allegations', 'allege', 'allow', 'ambulance',
       'amid', 'andrew', 'anger', 'angry', 'animal', 'announce', 'answer',
       'anti', 'anzac', 'apologise', 'appeal', 'appoint', 'approval',
       'approve', 'april', 'army', 'arrest', 'arrive', 'asbestos', 'asia',
       'asian', 'assault', 'asylum', 'attack', 'attempt', 'august',
       'aussie', 'aussies', 'aust', 'australia', 'australian',
       'australians', 'australias', 'authorities', 'avoid', 'await',
       'award', 'away', 'baby', 'baghdad', 'bail', 'bali', 'bangladesh',
       'bank', 'base', 'bash', 'battle', 'beach', 'beat', 'beattie',
       'beef', 'begin', 'bendigo', 'benefit', 'best', 'better', 'bike',
       'bik

In [39]:
def get_topics(comp, fn, n=5):
    for idx, topic in enumerate(comp):
        print("Topic %d:" % (idx+1), [(fn[i], topic[i].round(2)) for i in topic.argsort()[:-n - 1:-1]])

In [40]:
get_topics(lda_model.components_, terms)

Topic 1: [('government', 8725.19), ('sydney', 8393.29), ('queensland', 7720.12), ('change', 5874.27), ('home', 5674.38)]
Topic 2: [('australia', 13691.08), ('australian', 11088.95), ('melbourne', 7528.43), ('world', 6707.7), ('south', 6677.03)]
Topic 3: [('death', 5935.06), ('interview', 5924.98), ('kill', 5851.6), ('jail', 4632.85), ('life', 4275.27)]
Topic 4: [('house', 6113.49), ('2016', 5488.19), ('state', 4923.41), ('brisbane', 4857.21), ('tasmania', 4610.97)]
Topic 5: [('court', 7542.74), ('attack', 6959.64), ('open', 5663.0), ('face', 5193.63), ('warn', 5115.01)]
Topic 6: [('market', 5545.86), ('rural', 5502.89), ('plan', 4828.71), ('indigenous', 4223.4), ('power', 3968.26)]
Topic 7: [('charge', 8428.8), ('election', 7561.63), ('adelaide', 6758.36), ('make', 5658.99), ('test', 5062.69)]
Topic 8: [('police', 12092.44), ('crash', 5281.14), ('drug', 4290.87), ('beat', 3257.58), ('rise', 2934.92)]
Topic 9: [('fund', 4693.03), ('labor', 4047.69), ('national', 4038.68), ('council', 40

In [1]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.5 MB/s eta 0:00:00


In [2]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import urllib.request
from sklearn import preprocessing
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [3]:
# 훈련 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorNLI/multinli.train.ko.tsv", filename="multinli.train.ko.tsv")
urllib.request.urlretrieve("https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorNLI/snli_1.0_train.ko.tsv", filename="snli_1.0_train.ko.tsv")

('snli_1.0_train.ko.tsv', <http.client.HTTPMessage at 0x788d81114cd0>)

In [4]:
# 검증 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorNLI/xnli.dev.ko.tsv", filename="xnli.dev.ko.tsv")

('xnli.dev.ko.tsv', <http.client.HTTPMessage at 0x788d811172e0>)

In [5]:
# 테스트 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorNLI/xnli.test.ko.tsv", filename="xnli.test.ko.tsv")

('xnli.test.ko.tsv', <http.client.HTTPMessage at 0x788d81117130>)

In [6]:
train_snli = pd.read_csv("snli_1.0_train.ko.tsv", sep='\t', quoting=3)
train_xnli = pd.read_csv("multinli.train.ko.tsv", sep='\t', quoting=3)
val_data = pd.read_csv("xnli.dev.ko.tsv", sep='\t', quoting=3)
test_data = pd.read_csv("xnli.test.ko.tsv", sep='\t', quoting=3)

In [7]:
train_snli.head()

,sentence1,sentence2,gold_label
0,말을 탄 사람이 고장난 비행기 위로 뛰어오른다.,한 사람이 경쟁을 위해 말을 훈련시키고 있다.,neutral
1,말을 탄 사람이 고장난 비행기 위로 뛰어오른다.,한 사람이 식당에서 오믈렛을 주문하고 있다.,contradiction
2,말을 탄 사람이 고장난 비행기 위로 뛰어오른다.,사람은 야외에서 말을 타고 있다.,entailment
3,카메라에 웃고 손을 흔드는 아이들,그들은 부모님을 보고 웃고 있다,neutral
4,카메라에 웃고 손을 흔드는 아이들,아이들이 있다,entailment


In [8]:
train_xnli.head()

,sentence1,sentence2,gold_label
0,개념적으로 크림 스키밍은 제품과 지리라는 두 가지 기본 차원을 가지고 있다.,제품과 지리학은 크림 스키밍을 작동시키는 것이다.,neutral
1,시즌 중에 알고 있는 거 알아? 네 레벨에서 다음 레벨로 잃어버리는 거야 브레이브스...,사람들이 기억하면 다음 수준으로 물건을 잃는다.,entailment
2,우리 번호 중 하나가 당신의 지시를 세밀하게 수행할 것이다.,우리 팀의 일원이 당신의 명령을 엄청나게 정확하게 실행할 것이다.,entailment
3,어떻게 아세요? 이 모든 것이 다시 그들의 정보다.,이 정보는 그들의 것이다.,entailment
4,"그래, 만약 네가 테니스화 몇 개를 사러 간다면, 나는 왜 그들이 100달러대에서 ...",테니스화의 가격은 다양하다.,neutral


In [9]:
# 결합 후 섞기
train_data = train_snli.append(train_xnli)
train_data = train_data.sample(frac=1)

<ipython-input-9-9b851af9b5d4>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_snli.append(train_xnli)


In [10]:
train_data.head()

,sentence1,sentence2,gold_label
494284,"안경을 쓰고 빨간색과 검은색 줄무늬 셔츠를 입은 남자, 뭔가를 적고 있다.",안경 쓴 남자가 뭔가를 적고 있다.,entailment
271775,울타리를 뛰어넘는 백마를 타는 라이더.,기수가 말에 울타리를 치고 있다,entailment
191823,한 무리의 사람들이 밖으로 걸어 나간다.,한 무리의 관중들이 거리에서 사고를 보고 있다.,neutral
452987,한 소녀와 한 여자가 거리를 걸어가서 전화 부스를 지나가려고 한다.,차 안의 한 소녀와 한 여성이 전화 부스를 지나가려고 한다.,contradiction
446723,분홍색 안경을 쓴 남자가 그를 향해 떠다니는 것을 본다.,한 남자가 집에 앉아 있다,contradiction
